In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import os
from google.colab import drive

def load_and_prepare_data(sheet_url):
    """Loads data from a Google Sheet, handling URL conversion and cleaning."""
    try:
        csv_url = sheet_url.replace('/edit?usp=sharing', '/export?format=csv')
        df = pd.read_csv(csv_url)
        # Clean numeric columns with comma as decimal separator
        for col in df.columns:
            if df[col].dtype == 'object':
                try:
                    df[col] = df[col].str.replace(',', '.').astype(float)
                except ValueError:
                    pass  # Ignore if conversion fails (likely a non-numeric column)
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def convert_categorical_variables(df):
    """Converts categorical variables to numerical using Label Encoding."""
    le = LabelEncoder()
    try:
        if 'Sexo' in df.columns:
            df['Sexo_encoded'] = le.fit_transform(df['Sexo'].astype(str))
        if 'Impulso_A_Pornografia' in df.columns:
            df['Impulso_A_Pornografia_encoded'] = le.fit_transform(df['Impulso_A_Pornografia'].astype(str))
    except Exception as e:
        print(f"Error encoding variables: {e}")
    return df

def analyze_demographics(df):
    """Analyze demographic characteristics of the sample."""
    try:
        age_stats = df['Idade'].describe()
        gender_dist = df['Sexo'].value_counts()
        impulse_dist = df['Impulso_A_Pornografia'].value_counts()
        return {
            'age_statistics': age_stats,
            'gender_distribution': gender_dist,
            'impulse_distribution': impulse_dist
        }
    except KeyError as e:
        print(f"Missing key in demographics analysis: {e}. Check column names.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during demographic analysis: {e}")
        return None

def perform_statistical_tests(df):
    """Perform relevant statistical tests."""
    results = {}
    try:
        if 'Sexo_encoded' in df.columns and 'Sociosexualidade_Geral' in df.columns:
            if 0 in df['Sexo_encoded'].unique() and 1 in df['Sexo_encoded'].unique():
                t_stat, p_val = stats.ttest_ind(
                    df[df['Sexo_encoded'] == 1]['Sociosexualidade_Geral'],
                    df[df['Sexo_encoded'] == 0]['Sociosexualidade_Geral']
                )
                results['gender_comparison'] = {'t_stat': t_stat, 'p_value': p_val}
            else:
                results['gender_comparison'] = "Not enough data for both genders."
        else:
            results['gender_comparison'] = "Required columns ('Sexo_encoded' or 'Sociosexualidade_Geral') missing."

        if 'Impulso_A_Pornografia_encoded' in df.columns and 'Sociosexualidade_Geral' in df.columns:
            groups = [group['Sociosexualidade_Geral'].values for name, group in df.groupby('Impulso_A_Pornografia_encoded')]
            if len(groups) > 1:
                f_stat, p_val_anova = stats.f_oneway(*groups)
                results['impulse_comparison'] = {'f_stat': f_stat, 'p_value': p_val_anova}
            else:
                results['impulse_comparison'] = "Not enough impulse categories for ANOVA."
        else:
            results['impulse_comparison'] = "Required columns ('Impulso_A_Pornografia_encoded' or 'Sociosexualidade_Geral') missing."

        required_cols = ['Sociosexualidade_Geral', 'Satisfacao_Individual', 'Satisfacao_Interpersonal', 'Funcao_Sexual']
        if all(col in df.columns for col in required_cols):
            correlation_matrix = df[required_cols].corr()
            results['correlations'] = correlation_matrix
        else:
            results['correlations'] = "One or more required columns for correlation analysis are missing."
    except Exception as e:
        print(f"Error during statistical testing: {e}")
        return None
    return results

def create_visualizations(df, output_dir):
    """Create visualizations and save them, with dark graphite background and neon colors."""

    # Setup plot style
    plt.style.use('dark_background')
    plt.rcParams.update({
        'figure.facecolor': '#333333',  # Dark graphite background
        'axes.facecolor': '#333333',
        'axes.edgecolor': '#00FFFF',  # Cyan
        'axes.labelcolor': '#00FF00',  # Neon green
        'text.color': '#FF00FF',  # Magenta
        'xtick.color': '#00FF00',
        'ytick.color': '#00FF00',
        'grid.color': '#00FFFF',
        'grid.alpha': 0.7
    })

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Helper function for plotting
    def save_plot(fig, filename):
        fig.savefig(os.path.join(output_dir, filename), bbox_inches='tight', pad_inches=0.1)
        plt.close(fig)

    # --- Visualization 1: Age Distribution by Gender ---
    if 'Sexo_encoded' in df.columns and 'Idade' in df.columns:
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.boxplot(x='Sexo_encoded', y='Idade', data=df, palette='cool', ax=ax)
        ax.set_title('Age Distribution by Gender', color='#FF00FF')
        ax.set_xlabel('Gender (Encoded)', color='#00FF00')
        ax.set_ylabel('Age', color='#00FF00')
        save_plot(fig, '1_age_distribution.png')

    # --- Visualization 2: Sociosexuality by Impulse Category ---
    if 'Impulso_A_Pornografia_encoded' in df.columns and 'Sociosexualidade_Geral' in df.columns:
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.violinplot(x='Impulso_A_Pornografia_encoded', y='Sociosexualidade_Geral', data=df, palette='spring', ax=ax)
        ax.set_title('Sociosexuality Distribution by Impulse Category', color='#FF00FF')
        ax.set_xlabel('Impulse Category (Encoded)', color='#00FF00')
        ax.set_ylabel('Sociosexuality', color='#00FF00')
        save_plot(fig, '2_sociosexuality_distribution.png')

    # --- Visualization 3: Correlation Heatmap ---
    required_cols = ['Sociosexualidade_Geral', 'Satisfacao_Individual', 'Satisfacao_Interpersonal', 'Funcao_Sexual']
    if all(col in df.columns for col in required_cols):
        fig, ax = plt.subplots(figsize=(12, 8))
        sns.heatmap(df[required_cols].corr(), annot=True, cmap='magma', fmt=".2f", linewidths=.5, ax=ax, cbar_kws={'label': 'Correlation'})
        ax.set_title('Correlation Heatmap of Key Variables', color='#FF00FF')
        save_plot(fig, '3_correlation_heatmap.png')

    # --- Visualization 4: Histograms of Key Variables ---
    for col in required_cols:
        if col in df.columns:
            fig, ax = plt.subplots(figsize=(8, 5))
            sns.histplot(df[col], kde=True, color='#00FFFF', ax=ax)  # Cyan histogram
            ax.set_title(f'Distribution of {col}', color='#FF00FF')
            ax.set_xlabel(col, color='#00FF00')
            ax.set_ylabel('Frequency', color='#00FF00')
            save_plot(fig, f'4_histogram_{col}.png')

    # --- Visualization 5: Scatter Plots of Key Variable Pairs ---
    for i in range(len(required_cols)):
        for j in range(i + 1, len(required_cols)):
            col1, col2 = required_cols[i], required_cols[j]
            if col1 in df.columns and col2 in df.columns:
                fig, ax = plt.subplots(figsize=(8, 5))
                sns.scatterplot(x=col1, y=col2, data=df, color='#FF00FF', ax=ax)  # Magenta scatter plot
                ax.set_title(f'{col1} vs. {col2}', color='#FF00FF')
                ax.set_xlabel(col1, color='#00FF00')
                ax.set_ylabel(col2, color='#00FF00')
                save_plot(fig, f'5_scatter_{col1}_vs_{col2}.png')

    # --- Visualization 6: Pairplot of Key Variables ---
    if all(col in df.columns for col in required_cols):
        pairplot = sns.pairplot(df[required_cols], diag_kind='kde', plot_kws={'color': '#00FFFF'}, diag_kws={'color': '#00FFFF'})
        pairplot.fig.suptitle('Pairplot of Key Variables', y=1.02, color='#FF00FF')
        pairplot.savefig(os.path.join(output_dir, '6_pairplot.png'))
        plt.close(pairplot.fig)

    # --- Visualization 7: Boxplots of Key Variables ---
    for col in required_cols:
        if col in df.columns:
            fig, ax = plt.subplots(figsize=(8, 5))
            sns.boxplot(y=df[col], color='#00FF00', ax=ax)  # Green boxplot
            ax.set_title(f'Boxplot of {col}', color='#FF00FF')
            ax.set_ylabel(col, color='#00FF00')
            save_plot(fig, f'7_boxplot_{col}.png')

    # --- Visualization 8: Countplot of Impulse Categories ---
    if 'Impulso_A_Pornografia' in df.columns:
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.countplot(x='Impulso_A_Pornografia', data=df, palette='viridis', ax=ax)
        ax.set_title('Count of Impulse Categories', color='#FF00FF')
        ax.set_xlabel('Impulse Category', color='#00FF00')
        ax.set_ylabel('Count', color='#00FF00')
        save_plot(fig, '8_countplot_impulse.png')

    # --- Visualization 9: Countplot of Gender ---
    if 'Sexo' in df.columns:
        fig, ax = plt.subplots(figsize=(8, 5))
        sns.countplot(x='Sexo', data=df, palette='plasma', ax=ax)
        ax.set_title('Count of Gender', color='#FF00FF')
        ax.set_xlabel('Gender', color='#00FF00')
        ax.set_ylabel('Count', color='#00FF00')
        save_plot(fig, '9_countplot_gender.png')

    # --- Visualization 10: Age Distribution (Histogram) ---
    if 'Idade' in df.columns:
        fig, ax = plt.subplots(figsize=(8, 5))
        sns.histplot(df['Idade'], kde=True, color='#FFFF00', ax=ax)  # Yellow histogram
        ax.set_title('Age Distribution', color='#FF00FF')
        ax.set_xlabel('Age', color='#00FF00')
        ax.set_ylabel('Frequency', color='#00FF00')
        save_plot(fig, '10_histogram_age.png')

def main():
    """Main function to execute the analysis."""
    sheet_url = 'https://docs.google.com/spreadsheets/d/1hZaJlH52vXv3fF-heki6vBwffYLebjDJhDj7QwXzDgM/edit?usp=sharing'
    output_directory = '/content/drive/MyDrive/data'

    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return

    df = load_and_prepare_data(sheet_url)
    if df is None:
        return

    df = convert_categorical_variables(df)

    demographics = analyze_demographics(df)
    if demographics:
        print("\nDemographic Analysis:")
        for key, value in demographics.items():
            print(f"\n{key}:\n{value}")

    statistics = perform_statistical_tests(df)
    if statistics:
        print("\nStatistical Test Results:")
        for key, value in statistics.items():
            print(f"\n{key}:\n{value}")

    create_visualizations(df, output_directory)
    print(f"\nVisualizations saved to: {output_directory}")

if __name__ == "__main__":
    main()

Mounted at /content/drive
Google Drive mounted successfully.

Demographic Analysis:

age_statistics:
count    40.000000
mean     22.400000
std       4.802777
min      18.000000
25%      20.000000
50%      21.000000
75%      23.000000
max      45.000000
Name: Idade, dtype: float64

gender_distribution:
Sexo
1    20
0    20
Name: count, dtype: int64

impulse_distribution:
Impulso_A_Pornografia
A    10
B    10
C    10
D    10
Name: count, dtype: int64

Statistical Test Results:

gender_comparison:
{'t_stat': -0.6339338769770295, 'p_value': 0.5299217289871763}

impulse_comparison:
{'f_stat': 0.623870150759333, 'p_value': 0.6042170652132962}

correlations:
                          Sociosexualidade_Geral  Satisfacao_Individual  \
Sociosexualidade_Geral                  1.000000               0.384442   
Satisfacao_Individual                   0.384442               1.000000   
Satisfacao_Interpersonal                0.241751               0.751663   
Funcao_Sexual                           

<ipython-input-1-d9d2adf6a9ad>:122: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Sexo_encoded', y='Idade', data=df, palette='cool', ax=ax)
<ipython-input-1-d9d2adf6a9ad>:131: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='Impulso_A_Pornografia_encoded', y='Sociosexualidade_Geral', data=df, palette='spring', ax=ax)
<ipython-input-1-d9d2adf6a9ad>:186: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='Impulso_A_Pornografia', data=df, palette='viridis', ax=ax)
<ipython-input-1-d9d2adf6a9ad>:195: FutureWarning: 

Passing `palette` witho


Visualizations saved to: /content/drive/MyDrive/data
